In [ ]:
!pip install spektral
import numpy as np
import tensorflow as tf
import spektral
import seaborn as sns
import pandas as pd
from matplotlib import pyplot as plt
sns.set()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.1/140.1 KB 2.5 MB/s eta 0:00:00


In [ ]:
cora_dataset = spektral.datasets.citation.Citation(name='cora')
test_mask = cora_dataset.mask_te
train_mask = cora_dataset.mask_tr
val_mask = cora_dataset.mask_va
graph = cora_dataset.graphs[0]
features = graph.x
adj = graph.a
labels = graph.y
adj=adj.todense()
adj = tf.cast(adj+np.eye(adj.shape[0]),dtype=tf.float32)
deg = tf.reduce_sum(adj,axis=-1)
norm_deg = tf.linalg.diag(1.0/tf.sqrt(deg))
norm_adj = tf.matmul(norm_deg,tf.matmul(adj,norm_deg))

/usr/local/lib/python3.9/dist-packages/scipy/sparse/_index.py:146: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


In [ ]:
def masked_softmax_cross_entropy(logits,labels,mask):
  loss = tf.nn.softmax_cross_entropy_with_logits(logits=logits,labels=labels)
  mask = tf.cast(mask,dtype=tf.float32)
  mask /= tf.reduce_mean(mask)
  loss*=mask
  return tf.reduce_mean(loss)

def masked_accuracy(logits,labels,mask):
  correct_prediction = tf.equal(tf.argmax(logits,1),tf.argmax(labels,1))
  accuracy_all = tf.cast(correct_prediction,tf.float32)
  mask = tf.cast(mask,dtype=tf.float32)
  mask /= tf.reduce_mean(mask)
  accuracy_all *= mask
  return tf.reduce_mean(accuracy_all)

In [ ]:
def gnn(fts,adj,transform,activation):
  seq_fts = transform(fts)
  ret_fts = tf.matmul(adj,seq_fts)
  return activation(ret_fts)
def gnn_kuramoto(theta,adj,k,omega,dt,activation,transform,gnn_fn):
  [N,F] = np.shape(theta)
  theta = theta + dt*(omega*np.ones((N,F)) + (k/N)*tf.matmul(adj,gnn_fn(np.sin(theta),adj,transform,activation)))
  return theta
def gnn_hopf_rk4(X,Y,adj,a,beta,omega,k,dt,activation,transform,gnn_fn):
  [N,F] = np.shape(X)
  xk1 = a*np.ones((N,F))*X + (np.power(X,2) + np.power(Y,2))*(beta*Y-X) - omega*Y + k*activation(gnn_fn(X,adj,transform,activation))
  yk1 = a*np.ones((N,F))*Y - (np.power(X,2) + np.power(Y,2))*(beta*Y+X) - omega*X + k*activation(gnn_fn(Y,adj,transform,activation))

  xk2 = a*np.ones((N,F))*(X+dt*0.5*xk1) + (np.power(X+dt*0.5*xk1,2) + np.power(Y+dt*0.5*yk1,2))*(beta*(Y+dt*0.5*yk1)-(X+dt*0.5*xk1)) - omega*(Y+dt*0.5*yk1) + k*activation(gnn_fn(X+dt*0.5*xk1,adj,transform,activation))
  yk2 = a*np.ones((N,F))*(Y+dt*0.5*yk1) - (np.power(X+dt*0.5*xk1,2) + np.power(Y+dt*0.5*yk1,2))*(beta*(Y+dt*0.5*yk1)+(X+dt*0.5*xk1)) - omega*(X+dt*0.5*xk1) + k*activation(gnn_fn(Y+dt*0.5*yk1,adj,transform,activation))

  xk3 = a*np.ones((N,F))*(X+dt*0.5*xk2) + (np.power(X+dt*0.5*xk2,2) + np.power(Y+dt*0.5*yk2,2))*(beta*(Y+dt*0.5*yk2)-(X+dt*0.5*xk2)) - omega*(Y+dt*0.5*yk2) + k*activation(gnn_fn(X+dt*0.5*xk2,adj,transform,activation))
  yk3 = a*np.ones((N,F))*(Y+dt*0.5*yk2) - (np.power(X+dt*0.5*xk2,2) + np.power(Y+dt*0.5*yk2,2))*(beta*(Y+dt*0.5*yk2)+(X+dt*0.5*xk2)) - omega*(X+dt*0.5*xk2) + k*activation(gnn_fn(Y+dt*0.5*yk2,adj,transform,activation))
  
  xk4 = a*np.ones((N,F))*(X+dt*xk3) + (np.power(X+dt*xk3,2) + np.power(Y+dt*yk3,2))*(beta*(Y+dt*yk3)-(X+dt*xk3)) - omega*(Y+dt*yk3) + k*activation(gnn_fn(X+dt*xk3,adj,transform,activation))
  yk4 = a*np.ones((N,F))*(Y+dt*yk3) - (np.power(X+dt*xk3,2) + np.power(Y+dt*yk3,2))*(beta*(Y+dt*yk3)+(X+dt*xk3)) - omega*(X+dt*xk3) + k*activation(gnn_fn(Y+dt*yk3,adj,transform,activation))
  
  X = X + (dt/6)*(xk1+2*xk2+2*xk3+xk4)
  Y = Y + (dt/6)*(yk1+2*yk2+2*yk3+yk4)
  return [X,Y]
def gnn_xy(theta,p,adj,J,k_avg,dt,activation,transform,gnn_fn):
  [N,F] = np.shape(theta)
  p = p + dt*(-J/k_avg)*tf.matmul(adj,np.sin(gnn_fn(theta,adj,transform,activation)))
  theta = theta + dt*p
  return [theta,p]
def orderparam(theta):
  [N,F] = np.shape(theta)
  KOP = np.zeros((N,F),dtype = complex)
  KOP = np.cos(theta) + 1j*np.sin(theta)
  return np.abs(((1/(N*F))*np.sum(KOP)))

In [ ]:
def train_cora_gcn(fts, adj, gnn_fn, units, epochs, lr):
  lyr_1 = tf.keras.layers.Dense(units)
  lyr_2 = tf.keras.layers.Dense(7)
  def cora_gnn(fts,adj):
    hidden = gnn_fn(fts,adj,lyr_1,tf.nn.relu)
    logits = gnn_fn(hidden,adj,lyr_2,tf.identity)
    return logits
  
  optimiser = tf.keras.optimizers.Adam(learning_rate=lr)

  best_accuracy = 0.0

  results = np.zeros((epochs+1,3))
  for ep in range(epochs +1):
    with tf.GradientTape() as t:
      logits = cora_gnn(fts,adj)
      loss = masked_softmax_cross_entropy(logits,labels,train_mask)
    
    variables = t.watched_variables()
    grads = t.gradient(loss,variables)
    optimiser.apply_gradients(zip(grads,variables))

    logits = cora_gnn(fts,adj)
    val_accuracy = masked_accuracy(logits,labels,val_mask)
    test_accuracy = masked_accuracy(logits,labels,test_mask)

    if val_accuracy > best_accuracy:
      results[ep,0] =  loss.numpy(); results[ep,1] = val_accuracy.numpy(); results[ep,2] = test_accuracy.numpy()
      print('Epoch',ep,' | Training Loss:', loss.numpy(), ' | Val accuracy:', val_accuracy.numpy(), ' | Test accuracy:', test_accuracy.numpy())
  
  return results

def train_cora_kuramoto(fts, adj, gnn_fn, units, epochs, lr, timesteps,dt,orderparam):
  inputlyr = tf.keras.layers.Dense(units)
  lyr = []
  for i in range(0,timesteps):
    lyr.append(tf.keras.layers.Dense(units))
  outputlyr = tf.keras.layers.Dense(7)
  def cora_gnn_kuramoto(fts,adj):
    omega =1; k =1;
    hidden = gnn_fn(fts,adj,inputlyr,tf.nn.relu)
    KOP = np.zeros(timesteps)
    for i in range(0,timesteps):
      hidden = gnn_kuramoto(hidden,adj,k,omega,dt,tf.nn.relu,lyr[i],gnn)
      KOP[i] = orderparam(hidden)
    logits = gnn_fn(hidden,adj,outputlyr,tf.identity)
    return [logits,KOP]
  
  optimiser = tf.keras.optimizers.Adam(learning_rate=lr)

  best_accuracy = 0.0
  results = np.zeros((epochs+1,3))
  OrderParamResults = np.zeros((epochs+1,timesteps))
  for ep in range(epochs +1):
    with tf.GradientTape() as t:
      [logits,KOP] = cora_gnn_kuramoto(fts,adj)
      loss = masked_softmax_cross_entropy(logits,labels,train_mask)
    
    variables = t.watched_variables()
    grads = t.gradient(loss,variables)
    optimiser.apply_gradients(zip(grads,variables))

    [logits,KOP] = cora_gnn_kuramoto(fts,adj)
    OrderParamResults[ep,:] = KOP;
    val_accuracy = masked_accuracy(logits,labels,val_mask)
    test_accuracy = masked_accuracy(logits,labels,test_mask)

    if val_accuracy > best_accuracy:
      results[ep,0] =  loss.numpy(); results[ep,1] = val_accuracy.numpy(); results[ep,2] = test_accuracy.numpy()      
      print('Epoch',ep,' | Training Loss:', loss.numpy(), ' | Val accuracy:', val_accuracy.numpy(), ' | Test accuracy:', test_accuracy.numpy())

  return [results,OrderParamResults]

def train_cora_hopf(fts, adj, gnn_fn, units, epochs, lr, timesteps,dt,orderparam):
  inputlyr = tf.keras.layers.Dense(units)
  lyr = []
  for i in range(0,timesteps):
    lyr.append(tf.keras.layers.Dense(units))
  outputlyr = tf.keras.layers.Dense(7)
  def cora_gnn_hopf(fts,adj):
    omega=1; k=1; a=-0.02; beta=0
    hidden = gnn_fn(fts,adj,inputlyr,tf.nn.relu)
    Y = np.random.normal(0,1,np.shape(hidden))
    KOP = np.zeros(timesteps)
    for i in range(0,timesteps):
      hidden,Y = gnn_hopf_rk4(hidden,Y,adj,a,beta,omega,k,dt,tf.nn.relu,lyr[i],gnn)
      KOP[i] = orderparam(np.arctan(Y/hidden))
      
    logits = gnn_fn(hidden,adj,outputlyr,tf.identity)
    return [logits,KOP]
  
  optimiser = tf.keras.optimizers.Adam(learning_rate=lr)

  best_accuracy = 0.0
  resultshopf = np.zeros((epochs+1,3))
  OrderParamResults = np.zeros((epochs+1,timesteps))

  for ep in range(epochs +1):
    with tf.GradientTape() as t:
      logits,KOP = cora_gnn_hopf(fts,adj)
      loss = masked_softmax_cross_entropy(logits,labels,train_mask)
    
    variables = t.watched_variables()
    grads = t.gradient(loss,variables)
    optimiser.apply_gradients(zip(grads,variables))

    logits,KOP = cora_gnn_hopf(fts,adj)
    OrderParamResults[ep,:] = KOP;
    val_accuracy = masked_accuracy(logits,labels,val_mask)
    test_accuracy = masked_accuracy(logits,labels,test_mask)

    if val_accuracy > best_accuracy:
      resultshopf[ep,0] =  loss.numpy(); resultshopf[ep,1] = val_accuracy.numpy(); resultshopf[ep,2] = test_accuracy.numpy()      
      print('Epoch',ep,' | Training Loss:', loss.numpy(), ' | Val accuracy:', val_accuracy.numpy(), ' | Test accuracy:', test_accuracy.numpy())

  return [resultshopf,OrderParamResults]

def train_cora_xy(fts, adj, k_avg, gnn_fn, units, epochs, lr, timesteps,dt,orderparam):
  inputlyr = tf.keras.layers.Dense(units)
  lyr = []
  for i in range(0,timesteps):
    lyr.append(tf.keras.layers.Dense(units))
  outputlyr = tf.keras.layers.Dense(7)
  def cora_gnn_xy(fts,adj):
    J=0.5
    hidden = gnn_fn(fts,adj,inputlyr,tf.nn.relu)
    T=0.2
    p = np.random.normal(0,T,np.shape(hidden))
    KOP = np.zeros(timesteps)
    for i in range(0,timesteps):
      hidden,p = gnn_xy(hidden,p,adj,J,k_avg,dt,tf.nn.relu,lyr[i],gnn)
      KOP[i] = orderparam(hidden)
    logits = gnn_fn(hidden,adj,outputlyr,tf.identity)
    return [logits,KOP]
  
  optimiser = tf.keras.optimizers.Adam(learning_rate=lr)

  best_accuracy = 0.0
  resultsxy = np.zeros((epochs+1,3))
  OrderParamResults = np.zeros((epochs+1,timesteps))

  for ep in range(epochs +1):
    with tf.GradientTape() as t:
      logits,KOP = cora_gnn_xy(fts,adj)
      loss = masked_softmax_cross_entropy(logits,labels,train_mask)
    
    variables = t.watched_variables()
    grads = t.gradient(loss,variables)
    optimiser.apply_gradients((grad, variable) for (grad, variable) in zip(grads, variables) if grad is not None)
    logits, KOP = cora_gnn_xy(fts,adj)
    OrderParamResults[ep,:] = KOP;
    val_accuracy = masked_accuracy(logits,labels,val_mask)
    test_accuracy = masked_accuracy(logits,labels,test_mask)

    if val_accuracy > best_accuracy:
      resultsxy[ep,0] =  loss.numpy(); resultsxy[ep,1] = val_accuracy.numpy(); resultsxy[ep,2] = test_accuracy.numpy()      
      print('Epoch',ep,' | Training Loss:', loss.numpy(), ' | Val accuracy:', val_accuracy.numpy(), ' | Test accuracy:', test_accuracy.numpy())

  return [resultsxy,OrderParamResults]

In [ ]:
results = train_cora_gcn(features,norm_adj,gnn,32,200,0.01)
np.savetxt("GCNLearning.csv", results, delimiter=",")

Epoch 0  | Training Loss: 1.932569  | Val accuracy: 0.42  | Test accuracy: 0.47300002
Epoch 1  | Training Loss: 1.7454284  | Val accuracy: 0.50200003  | Test accuracy: 0.531
Epoch 2  | Training Loss: 1.520106  | Val accuracy: 0.532  | Test accuracy: 0.55099994
Epoch 3  | Training Loss: 1.2885818  | Val accuracy: 0.584  | Test accuracy: 0.60699993
Epoch 4  | Training Loss: 1.0730174  | Val accuracy: 0.652  | Test accuracy: 0.66099995
Epoch 5  | Training Loss: 0.8776017  | Val accuracy: 0.696  | Test accuracy: 0.71799994
Epoch 6  | Training Loss: 0.707538  | Val accuracy: 0.73800004  | Test accuracy: 0.74200004
Epoch 7  | Training Loss: 0.5639036  | Val accuracy: 0.758  | Test accuracy: 0.76000005
Epoch 8  | Training Loss: 0.44440553  | Val accuracy: 0.77599996  | Test accuracy: 0.7799999
Epoch 9  | Training Loss: 0.3463936  | Val accuracy: 0.7839999  | Test accuracy: 0.79999995
Epoch 10  | Training Loss: 0.26777792  | Val accuracy: 0.7859999  | Test accuracy: 0.80100006
Epoch 11  | Trai

In [ ]:
resultskuramoto, kopkuramoto = train_cora_kuramoto(features,norm_adj,gnn,32,200,0.01,20,0.05,orderparam)
np.savetxt("KuramotoLearning.csv", resultskuramoto, delimiter=",")
np.savetxt("KuramotoKOP.csv", kopkuramoto, delimiter=",")

Epoch 0  | Training Loss: 2.4113662  | Val accuracy: 0.18199998  | Test accuracy: 0.19499998
Epoch 1  | Training Loss: 2.0506442  | Val accuracy: 0.35599998  | Test accuracy: 0.36999997
Epoch 2  | Training Loss: 1.8294983  | Val accuracy: 0.38000003  | Test accuracy: 0.36799997
Epoch 3  | Training Loss: 1.6111234  | Val accuracy: 0.496  | Test accuracy: 0.47699994
Epoch 4  | Training Loss: 1.397306  | Val accuracy: 0.616  | Test accuracy: 0.608
Epoch 5  | Training Loss: 1.1982341  | Val accuracy: 0.6639999  | Test accuracy: 0.66499996
Epoch 6  | Training Loss: 1.0369309  | Val accuracy: 0.542  | Test accuracy: 0.559
Epoch 7  | Training Loss: 0.91466314  | Val accuracy: 0.566  | Test accuracy: 0.58199996
Epoch 8  | Training Loss: 0.7908541  | Val accuracy: 0.67399997  | Test accuracy: 0.67399997
Epoch 9  | Training Loss: 0.65850526  | Val accuracy: 0.74799997  | Test accuracy: 0.75899994
Epoch 10  | Training Loss: 0.5398106  | Val accuracy: 0.778  | Test accuracy: 0.7929999
Epoch 11  | 

In [ ]:
resultshopf, kophopf = train_cora_hopf(features,norm_adj,gnn,32,200,0.01,100,0.001,orderparam)
np.savetxt("HopfLearning.csv", resultshopf, delimiter=",")
np.savetxt("HopfKOP.csv", kophopf, delimiter=",")

Epoch 0  | Training Loss: 1.9510132  | Val accuracy: 0.35399997  | Test accuracy: 0.36799997
Epoch 1  | Training Loss: 1.7571118  | Val accuracy: 0.44399998  | Test accuracy: 0.50799996
Epoch 2  | Training Loss: 1.5479128  | Val accuracy: 0.57799995  | Test accuracy: 0.621
Epoch 3  | Training Loss: 1.316451  | Val accuracy: 0.668  | Test accuracy: 0.69499993
Epoch 4  | Training Loss: 1.0998149  | Val accuracy: 0.72400004  | Test accuracy: 0.738
Epoch 5  | Training Loss: 0.91819847  | Val accuracy: 0.752  | Test accuracy: 0.746
Epoch 6  | Training Loss: 0.75540966  | Val accuracy: 0.76399994  | Test accuracy: 0.76299995
Epoch 7  | Training Loss: 0.6230428  | Val accuracy: 0.77599996  | Test accuracy: 0.77599996
Epoch 8  | Training Loss: 0.50119025  | Val accuracy: 0.77599996  | Test accuracy: 0.777
Epoch 9  | Training Loss: 0.40815082  | Val accuracy: 0.76399994  | Test accuracy: 0.779
Epoch 10  | Training Loss: 0.33283156  | Val accuracy: 0.7799999  | Test accuracy: 0.7810001
Epoch 11 

In [ ]:
np.savetxt("HopfLearning.csv", resultshopf, delimiter=",")
np.savetxt("HopfKOP.csv", kophopf, delimiter=",")

In [ ]:
resultsxy,kopxy = train_cora_xy(features,norm_adj,sum(sum(adj))/int(np.shape(adj)[0]),gnn,32,200,0.01,100,0.01,orderparam)


Epoch 0  | Training Loss: 1.9436587  | Val accuracy: 0.372  | Test accuracy: 0.38999996
Epoch 1  | Training Loss: 1.7427444  | Val accuracy: 0.58199996  | Test accuracy: 0.60999995
Epoch 2  | Training Loss: 1.5268403  | Val accuracy: 0.678  | Test accuracy: 0.686
Epoch 3  | Training Loss: 1.288581  | Val accuracy: 0.72999996  | Test accuracy: 0.747
Epoch 4  | Training Loss: 1.0887837  | Val accuracy: 0.756  | Test accuracy: 0.774
Epoch 5  | Training Loss: 0.87794214  | Val accuracy: 0.7639999  | Test accuracy: 0.7849999
Epoch 6  | Training Loss: 0.6991584  | Val accuracy: 0.782  | Test accuracy: 0.79600006
Epoch 7  | Training Loss: 0.54326034  | Val accuracy: 0.77000004  | Test accuracy: 0.79499996
Epoch 8  | Training Loss: 0.42106605  | Val accuracy: 0.76399994  | Test accuracy: 0.805
Epoch 9  | Training Loss: 0.3245651  | Val accuracy: 0.7859999  | Test accuracy: 0.79399997
Epoch 10  | Training Loss: 0.24898303  | Val accuracy: 0.78  | Test accuracy: 0.79499996
Epoch 11  | Training L

NameError: ignored

In [ ]:
https://drive.google.com/drive/folders/1M5AZBP0_MNghgNL7m3KbbspZCrWFCxwe?usp=share_link

In [ ]:
np.savetxt("XYLearning.csv", resultsxy, delimiter=",")
np.savetxt("XYKOP.csv", kopxy, delimiter=",")

NameError: ignored

In [ ]:
!pip install --upgrade gupload

from pydrive.auth import GoogleAuth
from google.colab import auth

# Authenticate and create the PyDrive client.
auth.authenticate_user()

KL = np.savetxt("KuramotoLearning.csv", resultskuramoto, delimiter=",")
KKOP = np.savetxt("KuramotoKOP.csv", kopkuramoto, delimiter=",")

!gupload --to '1M5AZBP0_MNghgNL7m3KbbspZCrWFCxwe' KuramotoLearning.csv

In [ ]:
resultshopf

array([[1.95101321e+00, 3.53999972e-01, 3.67999971e-01],
       [1.75711179e+00, 4.43999976e-01, 5.07999957e-01],
       [1.54791284e+00, 5.77999949e-01, 6.20999992e-01],
       [1.31645095e+00, 6.67999983e-01, 6.94999933e-01],
       [1.09981489e+00, 7.24000037e-01, 7.37999976e-01],
       [9.18198466e-01, 7.51999974e-01, 7.45999992e-01],
       [7.55409658e-01, 7.63999939e-01, 7.62999952e-01],
       [6.23042822e-01, 7.75999963e-01, 7.75999963e-01],
       [5.01190245e-01, 7.75999963e-01, 7.77000010e-01],
       [4.08150822e-01, 7.63999939e-01, 7.78999984e-01],
       [3.32831562e-01, 7.79999912e-01, 7.81000078e-01],
       [2.75194705e-01, 7.77999997e-01, 7.85999894e-01],
       [2.24315554e-01, 7.69999981e-01, 7.79999912e-01],
       [1.87649474e-01, 7.75999963e-01, 7.78999925e-01],
       [1.55333251e-01, 7.58000016e-01, 7.73999870e-01],
       [1.27935916e-01, 7.67999947e-01, 7.87999928e-01],
       [1.09015435e-01, 7.67999947e-01, 7.82999933e-01],
       [9.44583938e-02, 7.67999

In [ ]:
%cd

/root
